In [1]:
from Networks import MLP2
from Trainers import NavierStokes_2D_Trainer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# 初始化模型
input_shape = (2,)  # x,y
output_shape = (3,)  # u,v,p

model = MLP2(input_shape, output_shape, hidden_layers=[64, 64, 64, 64, 64], activation_function="tanh")

In [3]:
nu = 0.01
optimizer = optim.Adam(model.parameters(), lr=1e-3)

trainer = NavierStokes_2D_Trainer(model, nu, optimizer, lambda_pde=1e-3, device='cuda')

In [4]:
# 定义网格范围
x = torch.linspace(0, 2, 64)  # x 范围从 0 到 2，共 64 个点
y = torch.linspace(-0.1, 0.1, 16)  # y 范围从 -0.1 到 0.1，共 16 个点

# 使用 meshgrid 创建 2D 网格
x_grid, y_grid = torch.meshgrid(x, y, indexing='ij')  # 'ij' 保证顺序为 (x, y)

# 将网格展平为一个二维张量 (N, 2)，每一行代表一个 (x, y) 坐标
collocation_points = torch.stack([x_grid.flatten(), y_grid.flatten()], dim=1)

# 生成边界点: y=0.1 和 y=-0.1 时的 (x, y) 点
boundary_points_upper = torch.stack([x, torch.full_like(x, 0.1)], dim=1)  # y = 0.1
boundary_points_lower = torch.stack([x, torch.full_like(x, -0.1)], dim=1)  # y = -0.1

# 中心参考值
boundary_points_midline = torch.stack([x, torch.full_like(x, 0.0)], dim=1)  # y = 0

# 生成入口点：x = 0 时，y = linspace(-0.1, 0.1, 16) 对应的 (x, y) 点
boundary_points_inlet = torch.stack([torch.full_like(y, 0.0), y], dim=1)  # x = 0, y 范围

# 合并为边界点
boundary_points = torch.cat([boundary_points_upper, boundary_points_lower, boundary_points_midline, boundary_points_inlet], dim=0)
# 边界条件，u, v 在 y=±0.1 处都为 0；在 x=0 时，u=5.5, v=0, p=None
boundary_values_u = torch.zeros_like(boundary_points[:, 0])  # u = 0
boundary_values_v = torch.zeros_like(boundary_points[:, 0])  # v = 0
boundary_values_u[-80:-16] = 8.25
boundary_values_u[-16:] = 5.5
boundary_values_u = boundary_values_u.unsqueeze(1)  # 变为 (batch_size, 1)
boundary_values_v = boundary_values_v.unsqueeze(1)
boundary_values_p = None

In [5]:
# 开始训练
trainer.train(collocation_points, boundary_points, boundary_values_u, boundary_values_v, boundary_values_p, epochs=10000, save_path='model/PINN2.pth')

Training Progress:   0%|          | 0/10000 [00:00<?, ?it/s]

/home/whw/miniconda3/envs/jupyter_env/lib/python3.9/site-packages/torch/autograd/graph.py:769: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/conda/conda-bld/pytorch_1724789143830/work/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/10000, Total Loss: 2.3425e+01 | PDE Loss: 2.7495e-03 | BC Loss: 2.3425e+01
Epoch 101/10000, Total Loss: 1.4458e+01 | PDE Loss: 1.9951e-01 | BC Loss: 1.4457e+01
Epoch 201/10000, Total Loss: 1.4188e+01 | PDE Loss: 5.1579e+00 | BC Loss: 1.4183e+01
Epoch 301/10000, Total Loss: 8.9327e+00 | PDE Loss: 4.3579e+02 | BC Loss: 8.4969e+00
Epoch 401/10000, Total Loss: 7.8549e-01 | PDE Loss: 2.5713e+02 | BC Loss: 5.2835e-01
Epoch 501/10000, Total Loss: 6.0713e-01 | PDE Loss: 1.9493e+02 | BC Loss: 4.1219e-01
Epoch 601/10000, Total Loss: 5.4446e-01 | PDE Loss: 1.9300e+02 | BC Loss: 3.5146e-01
Epoch 701/10000, Total Loss: 5.1998e-01 | PDE Loss: 1.8716e+02 | BC Loss: 3.3283e-01
Epoch 801/10000, Total Loss: 5.0418e-01 | PDE Loss: 1.8586e+02 | BC Loss: 3.1832e-01
Epoch 901/10000, Total Loss: 5.0078e-01 | PDE Loss: 1.8433e+02 | BC Loss: 3.1645e-01
Epoch 1001/10000, Total Loss: 4.9773e-01 | PDE Loss: 1.8278e+02 | BC Loss: 3.1496e-01
Epoch 1101/10000, Total Loss: 4.7866e-01 | PDE Loss: 1.8450e+02 | 

KeyboardInterrupt: 

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

trainer = NavierStokes_2D_Trainer(model, nu, optimizer, lambda_pde=1e-7, device='cuda')
trainer.train(collocation_points, boundary_points, boundary_values_u, boundary_values_v, boundary_values_p, epochs=10000, save_path='model/PINN2.pth')